# Определение текущего уровня потребительской лояльности (NPS) с использованием Tableau

**Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.<br>
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».<br>
Компания провела опрос и попросила нас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.<br>
Чтобы оценить результаты опроса, оценки обычно делят на три группы:<br>
9-10 баллов — «cторонники» (англ. promoters);<br>
7-8 баллов — «нейтралы» (англ. passives);<br>
0-6 баллов — «критики» (англ. detractors).<br>
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».<br>
Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике.<br>
Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса.**

**Проект в Tableau:** <https://public.tableau.com/views/NPS_16878640782230/NPS_3?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link>

In [6]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [7]:
path_to_db_local = r'C:\Users\ekate\OneDrive\Документы\Проекты по Анализу данных\Определение уровня потребительской лояльности (NPS)\telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [8]:
query = """
WITH

location_info AS (SELECT country,
                  city,
                  location_id
                  FROM location),
                  
age_info AS (SELECT title AS age_segment,
             age_gr_id
             FROM age_segment),
             
traffic_info AS (SELECT title AS traffic_segment,
                 tr_gr_id
                 FROM traffic_segment),
                 
lifetime_info AS (SELECT title AS lifetime_segment,
                  lt_gr_id
                  FROM lifetime_segment)
                  
                  
SELECT u.user_id,
       u.lt_day,
         CASE
           WHEN lt_day <= 365 THEN 'да'
           WHEN lt_day > 365 THEN 'нет'
         END AS is_new,
       u.age,
       CAST(u.gender_segment AS varchar) AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       u.nps_score,
          CASE 
            WHEN nps_score >= 9 THEN 'сторонник'
            WHEN nps_score = 8 OR nps_score = 7 THEN 'нейтрал'
            WHEN nps_score <= 6 THEN 'критик'
          END AS nps_group,
      location_info.country,
      location_info.city,
      age_info.age_segment,
      traffic_info.traffic_segment,
      lifetime_info.lifetime_segment
FROM user AS u
LEFT OUTER JOIN location_info ON u.location_id=location_info.location_id
LEFT OUTER JOIN age_info ON u.age_gr_id=age_info.age_gr_id
LEFT OUTER JOIN traffic_info ON u.tr_gr_id=traffic_info.tr_gr_id
LEFT OUTER JOIN lifetime_info ON u.lt_gr_id=lifetime_info.lt_gr_id;
"""

In [9]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,nps_score,nps_group,country,city,age_segment,traffic_segment,lifetime_segment
0,A001A2,2320,нет,45.0,1.0,ANDROID,SMARTPHONE,10,сторонник,Россия,Уфа,05 45-54,04 1-5,08 36+
1,A001WF,2344,нет,53.0,0.0,ANDROID,SMARTPHONE,10,сторонник,Россия,Киров,05 45-54,04 1-5,08 36+
2,A003Q7,467,нет,57.0,0.0,ANDROID,SMARTPHONE,10,сторонник,Россия,Москва,06 55-64,08 20-25,06 13-24
3,A004TB,4190,нет,44.0,1.0,IOS,SMARTPHONE,10,сторонник,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+
4,A004XT,1163,нет,24.0,0.0,ANDROID,SMARTPHONE,10,сторонник,Россия,Рязань,02 16-24,05 5-10,08 36+
5,A005O0,5501,нет,42.0,1.0,ANDROID,SMARTPHONE,6,критик,Россия,Омск,04 35-44,05 5-10,08 36+
6,A0061R,1236,нет,45.0,0.0,ANDROID,SMARTPHONE,10,сторонник,Россия,Уфа,05 45-54,06 10-15,08 36+
7,A009KS,313,да,35.0,0.0,ANDROID,SMARTPHONE,10,сторонник,Россия,Москва,04 35-44,13 45-50,05 7-12
8,A00AES,3238,нет,36.0,1.0,ANDROID,SMARTPHONE,10,сторонник,Россия,СанктПетербург,04 35-44,04 1-5,08 36+
9,A00F70,4479,нет,54.0,1.0,ANDROID,SMARTPHONE,9,сторонник,Россия,Волгоград,05 45-54,07 15-20,08 36+


In [10]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Дашборд 1 "Исследование уровня потребительской лояльности"** <https://public.tableau.com/app/profile/.67731454/viz/NPS_16878640782230/sheet10?publish=yes>

**Дашборд 2 "Определение соотношения критиков и сторонников"** <https://public.tableau.com/app/profile/.67731454/viz/NPS_16878640782230/sheet11?publish=yes>

**Презентация в Tableau** <https://public.tableau.com/app/profile/.67731454/viz/NPS_16878640782230/NPS_3?publish=yes>